In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split

In [2]:
file_names = os.listdir('crop_part1')

In [3]:
import re
pattern = re.compile(r"(\d+)_(\d+)_(\d+)")

df = pd.DataFrame(columns = ['Age', 'Gender', 'Ethnicity'])
count = 0
for i in file_names:
    data =list(pattern.search(i).groups())
    df.loc[count] = data
    count += 1
        
    
#7881 and 7876 are bad values
# AGE, Gender, Ethnicity
# 0 is male, 1 is female
# Race is an integer from 0 to 4, denoting White, Black, Asian, Indian, and Others (like Hispanic, Latino, Middle Eastern).
# 0 = White
# 1 = Black
# 2 = Asian
# 3 = Indian
# 4 = Others

df['JPG'] = file_names
img_path = []
count = 0
for row in df.iterrows():
    path = "crop_part1/"+df.JPG.iloc[count]
    img_path.append(path)
    count += 1
df['img_path'] = img_path
df2 = df[['Age', 'Gender', 'Ethnicity', 'JPG', 'img_path']].copy()
df2.head()

,Age,Gender,Ethnicity,JPG,img_path
0,100,0,0,100_0_0_20170112213500903.jpg.chip.jpg,crop_part1/100_0_0_20170112213500903.jpg.chip.jpg
1,100,0,0,100_0_0_20170112215240346.jpg.chip.jpg,crop_part1/100_0_0_20170112215240346.jpg.chip.jpg
2,100,1,0,100_1_0_20170110183726390.jpg.chip.jpg,crop_part1/100_1_0_20170110183726390.jpg.chip.jpg
3,100,1,0,100_1_0_20170112213001988.jpg.chip.jpg,crop_part1/100_1_0_20170112213001988.jpg.chip.jpg
4,100,1,0,100_1_0_20170112213303693.jpg.chip.jpg,crop_part1/100_1_0_20170112213303693.jpg.chip.jpg


In [4]:
df3 = pd.DataFrame(columns = ['Age_Range'])
ages = df2['Age'].tolist()
count = 0
for i in ages:
    i = int(i)
    if i == 0 or i == 1 or i == 2 or i == 3:
        df3.loc[count] = '0-2'
    elif i == 4 or i == 5 or i == 6 or i == 7:
        df3.loc[count] = '4-6'
    elif i == 8 or i == 9 or i == 10 or i == 11 or i == 12 or i == 13 or i == 14:
        df3.loc[count] = '8-13'
    elif i == 15 or i == 16 or i == 17 or i == 18 or i == 19 or i == 20 or i == 21 or i == 22 or i == 23 or i == 24:
        df3.loc[count] = '15-20'
    elif i == 25 or i == 26 or i == 27 or i == 28 or i == 29 or i == 30 or i == 31 or i == 32 or i == 33 or i == 34 or i == 35 or i == 36 or i == 37 :
        df3.loc[count] = '25-32'
    elif i == 38 or i == 39 or i == 40 or i == 41 or i == 42 or i == 43 or i == 44 or i == 45 or i == 46 or i == 47:
        df3.loc[count] = '38-43'
    elif i == 48 or i == 49 or i == 50 or i == 51 or i == 52 or i == 53 or i == 54 or i == 55 or i == 56 or i == 57 or i == 58 or i == 59:
        df3.loc[count] = '48-53'
    else: 
        df3.loc[count] = '60+'
    count += 1
df3['Age_Range'].value_counts()

25-32    8980
15-20    3252
60+      2690
48-53    2600
38-43    2535
0-2      1894
8-13     1018
4-6       739
Name: Age_Range, dtype: int64

In [5]:
df4 = pd.concat([df2, df3], axis=1, join='inner')
age_to_label_map = {
    '0-2'  :0,
    '4-6'  :1,
    '8-13' :2,
    '15-20':3,
    '25-32':4,
    '38-43':5,
    '48-53':6,
    '60+'  :7
}
df4['Age_Range_Num'] = df4['Age_Range'].apply(lambda Age_Range: age_to_label_map[Age_Range])
df4.head()

,Age,Gender,Ethnicity,JPG,img_path,Age_Range,Age_Range_Num
0,100,0,0,100_0_0_20170112213500903.jpg.chip.jpg,crop_part1/100_0_0_20170112213500903.jpg.chip.jpg,60+,7
1,100,0,0,100_0_0_20170112215240346.jpg.chip.jpg,crop_part1/100_0_0_20170112215240346.jpg.chip.jpg,60+,7
2,100,1,0,100_1_0_20170110183726390.jpg.chip.jpg,crop_part1/100_1_0_20170110183726390.jpg.chip.jpg,60+,7
3,100,1,0,100_1_0_20170112213001988.jpg.chip.jpg,crop_part1/100_1_0_20170112213001988.jpg.chip.jpg,60+,7
4,100,1,0,100_1_0_20170112213303693.jpg.chip.jpg,crop_part1/100_1_0_20170112213303693.jpg.chip.jpg,60+,7


In [6]:
gender_df = pd.concat([df4,pd.get_dummies(df4['Gender'])],axis = 1)
gender_df = gender_df[['JPG','0','1']]
gender_df.columns = ['JPG','Male','Female']
gender_df.to_csv('gender.csv')

age_df = pd.concat([df4,pd.get_dummies(df4['Age_Range'])],axis = 1)
age_df  = age_df[['JPG','0-2','8-13','15-20','25-32','38-43','4-6','48-53','60+']]
age_df.to_csv('age.csv')

ethnicity_df = pd.concat([df4,pd.get_dummies(df4['Ethnicity'])],axis = 1)
ethnicity_df  = ethnicity_df[['JPG','0','1','2','3','4']]
ethnicity_df.columns = ['JPG','White','Black','Asian','Indian','Other']
ethnicity_df.to_csv('ethnicity.csv')

# Rename Gender Files

In [7]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random

# seed random number generator
seed(1)
# define ratio of pictures to use for validation
src_directory = 'crop_part1/'
dataset_home = 'dataset_male_vs_females2/'
subdirs = ['male/', 'female/']
for subdir in subdirs:
    newdir = dataset_home + subdir 
    makedirs(newdir, exist_ok=True)
    
count = 0
for file in listdir(src_directory):
    src = src_directory + '/' + file
    if df2.iloc[count]['JPG'] == file and int(df2.iloc[count]['Gender']) == 0:
        dst = dataset_home + 'male/'  + file
        copyfile(src, dst)
    elif df2.iloc[count]['JPG'] == file and int(df2.iloc[count]['Gender']) == 1:
        dst = dataset_home + 'female/' + file
        copyfile(src, dst)
    count += 1

In [8]:
trainM_names = os.listdir('dataset_male_vs_females2/male')
trainF_names = os.listdir('dataset_male_vs_females2/female')
print('Train Male: ',len(trainM_names))
print('Train Female: ',len(trainF_names))
# Train Male:  4370
# Train Female:  5406

Train Male:  12391
Train Female:  11317


In [9]:
df2.Gender.value_counts()

0    12391
1    11317
Name: Gender, dtype: int64

# Rename Age Files

In [10]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random

# seed random number generator
seed(1)
# define ratio of pictures to use for validation
src_directory = 'crop_part1/'
dataset_home = 'dataset_age2/'
subdirs = ['0-2/', '4-6/', '8-13/', '15-20/', '25-32/', '38-43/', '48-53/', '60+/']
for subdir in subdirs:
    newdir = dataset_home + subdir 
    makedirs(newdir, exist_ok=True)

count = 0
for file in listdir(src_directory):
    src = src_directory + '/' + file
    if df4.iloc[count]['JPG'] == file and int(df4.iloc[count]['Age_Range_Num']) == 0:
        dst = dataset_home + '0-2/'  + file
        copyfile(src, dst)
    elif df4.iloc[count]['JPG'] == file and int(df4.iloc[count]['Age_Range_Num']) == 1:
        dst = dataset_home + '4-6/' + file
        copyfile(src, dst)
    elif df4.iloc[count]['JPG'] == file and int(df4.iloc[count]['Age_Range_Num']) == 2:
        dst = dataset_home + '8-13/' + file
        copyfile(src, dst)
    elif df4.iloc[count]['JPG'] == file and int(df4.iloc[count]['Age_Range_Num']) == 3:
        dst = dataset_home + '15-20/' + file
        copyfile(src, dst)
    elif df4.iloc[count]['JPG'] == file and int(df4.iloc[count]['Age_Range_Num']) == 4:
        dst = dataset_home + '25-32/' + file
        copyfile(src, dst)
    elif df4.iloc[count]['JPG'] == file and int(df4.iloc[count]['Age_Range_Num']) == 5:
        dst = dataset_home + '38-43/' + file
        copyfile(src, dst)
    elif df4.iloc[count]['JPG'] == file and int(df4.iloc[count]['Age_Range_Num']) == 6:
        dst = dataset_home + '48-53/' + file
        copyfile(src, dst)
    else:
        dst = dataset_home + '60+/' + file
        copyfile(src, dst)
        
    count+= 1

In [11]:
names0 = os.listdir('dataset_age2/0-2')
names1 = os.listdir('dataset_age2/4-6')
names2 = os.listdir('dataset_age2/8-13')
names3 = os.listdir('dataset_age2/15-20')
names4 = os.listdir('dataset_age2/25-32')
names5 = os.listdir('dataset_age2/38-43')
names6 = os.listdir('dataset_age2/48-53')
names7 = os.listdir('dataset_age2/60+')


print('0-2: ',len(names0))
print('4-6: ',len(names1))
print('8-13: ',len(names2))
print('15-20: ',len(names3))
print('25-32: ',len(names4))
print('38-43: ',len(names5))
print('48-53: ',len(names6))
print('60+: ',len(names7))
print('Total: ', str(len(names0)+len(names1)+len(names2)+len(names3)+len(names4)+len(names5)+len(names6)+len(names7)))

0-2:  1894
4-6:  739
8-13:  1018
15-20:  3252
25-32:  8980
38-43:  2535
48-53:  2600
60+:  2690
Total:  23708


In [12]:
df4.Age_Range_Num.value_counts()

4    8980
3    3252
7    2690
6    2600
5    2535
0    1894
2    1018
1     739
Name: Age_Range_Num, dtype: int64

# Rename Ethnicity Files

In [13]:
df2.Ethnicity.value_counts()

0                    10078
1                     4526
3                     3975
2                     3434
4                     1692
20170116174525125        1
20170109142408075        1
20170109150557335        1
Name: Ethnicity, dtype: int64

In [8]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random

# 0 = White
# 1 = Black
# 2 = Asian
# 3 = Indian
# 4 = Others

# seed random number generator
seed(1)
# define ratio of pictures to use for validation
src_directory = 'crop_part1/'
dataset_home = 'dataset_ethnicity/'
subdirs = ['White/', 'Black/', 'Asian/', 'Indian/', 'Others/']
for subdir in subdirs:
    newdir = dataset_home + subdir 
    makedirs(newdir, exist_ok=True)

count = 0
for file in listdir(src_directory):
    src = src_directory + '/' + file
    if df2.iloc[count]['JPG'] == file and int(df2.iloc[count]['Ethnicity']) == 0:
        dst = dataset_home + 'White/'  + file
        copyfile(src, dst)
    elif df2.iloc[count]['JPG'] == file and int(df2.iloc[count]['Ethnicity']) == 1:
        dst = dataset_home + 'Black/' + file
        copyfile(src, dst)
    elif df2.iloc[count]['JPG'] == file and int(df2.iloc[count]['Ethnicity']) == 2:
        dst = dataset_home + 'Asian/' + file
        copyfile(src, dst)
    elif df2.iloc[count]['JPG'] == file and int(df2.iloc[count]['Ethnicity']) == 3:
        dst = dataset_home + 'Indian/' + file
        copyfile(src, dst)
    else:
        dst = dataset_home + 'Others/' + file
        copyfile(src, dst)
    count += 1

In [9]:
W_names = os.listdir('dataset_Ethnicity/White')
B_names = os.listdir('dataset_Ethnicity/Black')
A_names = os.listdir('dataset_Ethnicity/Asian')
I_names = os.listdir('dataset_Ethnicity/Indian')
O_names = os.listdir('dataset_Ethnicity/Others')


print('White: ',len(W_names))
print('Black: ',len(B_names))
print('Asian: ',len(A_names))
print('Indian: ',len(I_names))
print('Others: ',len(O_names))
print('Total: ', str(len(W_names)+len(B_names)+len(A_names)+len(I_names)+len(O_names)))


White:  10078
Black:  4526
Asian:  3434
Indian:  3975
Others:  1695
Total:  23708


In [ ]:
# White:  5264
# Black:  405
# Asian:  1553
# Indian:  1451
# Others:  1103
# Total:  9776

# Split files into train, test and val
## Gender

In [10]:
import os
import numpy as np
import shutil
import pandas as pd

def train_test_split(root_dir, classes_dir, processed_dir):
    print("########### Train Test Val Script started ###########")
    #data_csv = pd.read_csv("DataSet_Final.csv") ##Use if you have classes saved in any .csv file

    val_ratio = 0.20
    test_ratio = 0.20

    for cls in classes_dir:
        # Creating partitions of the data after shuffeling
        print("$$$$$$$ Class Name " + cls + " $$$$$$$")
        src = processed_dir +"/" + cls   # Folder to copy images from

        allFileNames = os.listdir(src)
        np.random.shuffle(allFileNames)
        train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                                  [int(len(allFileNames) * (1 - (val_ratio + test_ratio))),
                                                                   int(len(allFileNames) * (1 - val_ratio)),
                                                                   ])

        train_FileNames = [src + '//' + name for name in train_FileNames.tolist()]
        val_FileNames = [src + '//' + name for name in val_FileNames.tolist()]
        test_FileNames = [src + '//' + name for name in test_FileNames.tolist()]

        print('Total images: '+ str(len(allFileNames)))
        print('Training: '+ str(len(train_FileNames)))
        print('Validation: '+  str(len(val_FileNames)))
        print('Testing: '+ str(len(test_FileNames)))

        # # Creating Train / Val / Test folders (One time use)
        os.makedirs(root_dir + '/train//' + cls)
        os.makedirs(root_dir + '/val//' + cls )
        os.makedirs(root_dir + '/test//' + cls)
        # Copy-pasting images
        for name in train_FileNames:
            shutil.copy(name, root_dir + '/train//' + cls)

        for name in val_FileNames:
            shutil.copy(name, root_dir + '/val//' + cls)

        for name in test_FileNames:
            shutil.copy(name, root_dir + '/test//' + cls)

    print("########### Train Test Val Script Ended ###########")



In [18]:
new_direct = 'Gender_Images2'
classes = ['male', 'female']
processed_direct = 'dataset_male_vs_females2/'
train_test_split(new_direct, classes, processed_direct)

########### Train Test Val Script started ###########
$$$$$$$ Class Name male $$$$$$$
Total images: 12391
Training: 7434
Validation: 2478
Testing: 2479
$$$$$$$ Class Name female $$$$$$$
Total images: 11317
Training: 6790
Validation: 2263
Testing: 2264
########### Train Test Val Script Ended ###########


## Age

In [19]:
new_direct2 = 'Age_Images2'
classes2 = ['0-2/', '4-6/', '8-13/', '15-20/', '25-32/', '38-43/', '48-53/', '60+/']
processed_direct2 = 'dataset_age2/'
train_test_split(new_direct2, classes2, processed_direct2)

########### Train Test Val Script started ###########
$$$$$$$ Class Name 0-2/ $$$$$$$
Total images: 1894
Training: 1136
Validation: 379
Testing: 379
$$$$$$$ Class Name 4-6/ $$$$$$$
Total images: 739
Training: 443
Validation: 148
Testing: 148
$$$$$$$ Class Name 8-13/ $$$$$$$
Total images: 1018
Training: 610
Validation: 204
Testing: 204
$$$$$$$ Class Name 15-20/ $$$$$$$
Total images: 3252
Training: 1951
Validation: 650
Testing: 651
$$$$$$$ Class Name 25-32/ $$$$$$$
Total images: 8980
Training: 5388
Validation: 1796
Testing: 1796
$$$$$$$ Class Name 38-43/ $$$$$$$
Total images: 2535
Training: 1521
Validation: 507
Testing: 507
$$$$$$$ Class Name 48-53/ $$$$$$$
Total images: 2600
Training: 1560
Validation: 520
Testing: 520
$$$$$$$ Class Name 60+/ $$$$$$$
Total images: 2690
Training: 1614
Validation: 538
Testing: 538
########### Train Test Val Script Ended ###########


# Ethnicity

In [11]:
# Race is an integer from 0 to 4, denoting White, Black, Asian, Indian, and Others (like Hispanic, Latino, Middle Eastern).
# 0 = White
# 1 = Black
# 2 = Asian
# 3 = Indian
# 4 = Others

new_direct3 = 'Ethnicity_Images'
classes3 = ['White','Black','Asian','Indian','Others']
processed_direct3 = 'dataset_ethnicity/'

train_test_split(new_direct3, classes3, processed_direct3)

########### Train Test Val Script started ###########
$$$$$$$ Class Name White $$$$$$$
Total images: 10078
Training: 6046
Validation: 2016
Testing: 2016
$$$$$$$ Class Name Black $$$$$$$
Total images: 4526
Training: 2715
Validation: 905
Testing: 906
$$$$$$$ Class Name Asian $$$$$$$
Total images: 3434
Training: 2060
Validation: 687
Testing: 687
$$$$$$$ Class Name Indian $$$$$$$
Total images: 3975
Training: 2385
Validation: 795
Testing: 795
$$$$$$$ Class Name Others $$$$$$$
Total images: 1695
Training: 1017
Validation: 339
Testing: 339
########### Train Test Val Script Ended ###########
